In [1]:
import os
from pathlib import Path

testfolder = 'Simulations/1axis'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Simulations/1axis


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 21.6.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+243.gfdab617.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'bifacialvf_1axis'

# Location
lat = 39.7555
lon = -105.2211

# Scene Parameters
azimuth_ang=90 # Facing south
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 2 # m
module_y = 1 # m. slope we will measure
sensorsy=2
sensorsground=5

# SceneDict Parameters
pitch = 6 # m
albedo = 0.2
clearance_height = 0.5 # m  
nMods = 4 
nRows = 3

sceneDict = {'tilt':10,'pitch':3,'clearance_height':0.2,'azimuth':180, 'nMods': 20, 'nRows': 7} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)
epwfile = demo.getEPW(lat, lon)

path = Simulations/1axis
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: USA_CO_Golden-NREL.724666_TMY3.epw
 ... OK!


In [6]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y)


Module Name: PVmod
Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [7]:
hub_height = 1.5
pitch = 5.7
sazm = 180 # axis angle, N-S = 180
fixed_tilt_angle = None
gcr = 2 / pitch
cumulativesky = False
tracking = True

trackerParams = {
    'limit_angle': 50,
    'angledelta': 30,
    'backtrack': True,
    'gcr': gcr,
    'cumulativesky': cumulativesky,
    'azimuth': sazm,  # axis angle, N-S = 180
    'fixed_tilt_angle': fixed_tilt_angle
}

In [8]:
metdata = demo.readWeatherFile(epwfile, coerce_year=2021, starttime='2021-06-01_1100', endtime='2021-06-01_1300')

8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


In [9]:
trackerdict = demo.set1axis(**trackerParams)

In [10]:
trackerdict

{'2021-06-01_1100': {'surf_azm': 90.0,
  'surf_tilt': 21.2,
  'theta': -21.2,
  'dni': 0,
  'ghi': 181,
  'dhi': 181,
  'temp_air': 12.2,
  'wind_speed': 5.1},
 '2021-06-01_1200': {'surf_azm': 90.0,
  'surf_tilt': 6.94,
  'theta': -6.94,
  'dni': 0,
  'ghi': 229,
  'dhi': 229,
  'temp_air': 13.9,
  'wind_speed': 4.1},
 '2021-06-01_1300': {'surf_azm': 270.0,
  'surf_tilt': 7.6,
  'theta': 7.6,
  'dni': 585,
  'ghi': 884,
  'dhi': 331,
  'temp_air': 17.8,
  'wind_speed': 3.6}}

In [12]:
sceneDict = {
    'pitch': pitch,
    'hub_height': hub_height,
    'nMods': 5,
    'nRows': 2
}

In [13]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~3 .rad files for gendaylit 1-axis workflow (this takes a minute..)
3 Radfiles created in /objects/

Making 3 octfiles in root directory.
Trackerdict key error: 'skyfile'
Trackerdict key error: 'skyfile'
Trackerdict key error: 'skyfile'


In [14]:
trackerdict.keys()

dict_keys(['2021-06-01_1100', '2021-06-01_1200', '2021-06-01_1300'])

In [15]:
trackerdict

{'2021-06-01_1100': {'surf_azm': 90.0,
  'surf_tilt': 21.2,
  'theta': -21.2,
  'dni': 0,
  'ghi': 181,
  'dhi': 181,
  'temp_air': 12.2,
  'wind_speed': 5.1,
  'radfile': 'objects/1axis2021-06-01_1100__C_1.31919_rtr_5.70000_tilt_21.20000_5modsx2rows_origin0,0.rad',
  'scene': {'module': {'x': 2, 'y': 1, 'z': 0.02, 'modulematerial': 'black', 'scenex': 2.01, 'sceney': 1.0, 'scenez': 0.1, 'numpanels': 1, 'bifi': 1, 'text': '! genbox black PVmod 2 1 0.02 | xform -t -1.0 -0.5 0 -a 1 -t 0 1.0 0', 'modulefile': 'objects/PVmod.rad', 'glass': False, 'offsetfromaxis': 0, 'xgap': 0.01, 'ygap': 0.0, 'zgap': 0.1}, 'modulefile': 'objects/PVmod.rad', 'hpc': False, 'name': 'Scene0', 'gcr': 0.17543859649122806, 'text': '!xform -rx 21.2 -t 0 0 1.5 -a 5 -t 2.01 0 0 -a 2 -t 0 5.7 0 -i 1 -t -4.02 -0.0 0 -rz 90.0 -t 0 0 0 objects/PVmod.rad', 'radfiles': 'objects/1axis2021-06-01_1100__C_1.31919_rtr_5.70000_tilt_21.20000_5modsx2rows_origin0,0.rad', 'sceneDict': {'tilt': 21.2, 'pitch': 5.7, 'clearance_height'

In [16]:
trackerdict = demo.runViewFactor(sensorsy=4, tracking=True, backtrack=trackerParams['backtrack'], limit_angle=trackerParams['limit_angle'])

Calculating Sun position with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Albedo value passed, but also present in TMY3 file.  Using albedo value passed. To use the ones in TMY3 file re-run simulation with albedo=None

 
********* 
Running Simulation for TMY3: 
Location:   Denver Centennial  Golden   Nr
Lat:  39.74  Long:  -105.18  Tz  -7.0
Parameters: tilt:  0   Sazm:  90.0     Hub_Height :  1.5   Pitch:  5.7   Row type:  interior   Albedo:  0.15
Saving into bifacialvf_1axis
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.0
Actual distance between rows =  4.7
 
 ***** IMPORTANT --> THIS SIMULATION Has Tracking Activated
Backtracking Option is set to:  True


100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 155.03it/s]

Finished
